In [1]:
from sklearn import svm
import numpy as np
import os
import numpy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def SVM(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)

# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	

    
# SVM Classifier Valence
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    	
# SVM Classifier Arousal
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]


    


####################### Happy Sad ###################
def SVMHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    # We ignore the encoding of anything that isn't happy and sad , create a csv file
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
        
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]
###########All emotions #####################

def SVMCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

################ Run Each Method Multiple Times #####################
def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        svm=SVM(UserName,Window,StartTime,EndTime)
        arousal+=[svm[0]]
        valence+=[svm[1]]
        F1_ScoreArousal+=[svm[2]]
        F1_ScoreValence+=[svm[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=SVMCombined(UserName,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=SVMHS(UserName,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

##################Main#########################
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,33):
        print("s"+str(i))
        #Happy and Sad
        if not ("s"+str(i) == "s31"):
            var =RunHS100("s"+str(i))
            print("Happy Sad")
            print(var)
            HSAccurecy+=var[0]
            HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for 31 users "+str(HSAccurecy/31))
    print("F1 happy sad for  31 users "+str(HSF1/31))
    print("Accurecy Combined sad for 32 users"+str(CombinedAccurecy/32))
    print("F1 Combined sad for  32 users "+str(CombinedF1/32))
    print("Accurecy Valence for  32 users "+str(ValenceAccurecy/32))
    print("F1 Valence for 32 users "+str(ValenceF1/32))
    print("Accurecy Arousal for  32 users "+str(ArousalAccurecy/32))
    print("F1 Arousal for  32 users "+str(ArousalF1/32))


s01
Array of average Happy Sad accurecies for user s01
Counter({60.0: 45, 40.0: 32, 20.0: 12, 80.0: 9, 0.0: 2})
Happy Sad
[49.4, 0.4246825396825398]
Array of average All Emotions accurecies for user s01
Counter({37.5: 34, 25.0: 28, 50.0: 18, 12.5: 12, 62.5: 5, 0.0: 3})
Combined Emotions
[33.375, 0.2535568181818182]
Arousal 60.25
Counter({62.5: 29, 50.0: 27, 75.0: 20, 37.5: 10, 87.5: 8, 25.0: 3, 100.0: 2, 12.5: 1})
Valence 38.0
Counter({37.5: 44, 25.0: 23, 50.0: 22, 62.5: 7, 12.5: 3, 0.0: 1})
Arousal Valence
[60.25, 38.0, 0.488042513042513, 0.278989898989899]
s02
Array of average Happy Sad accurecies for user s02
Counter({100.0: 42, 80.0: 40, 60.0: 18})
Happy Sad
[84.8, 0.7854444444444444]
Array of average All Emotions accurecies for user s02
Counter({50.0: 32, 37.5: 31, 62.5: 16, 25.0: 9, 12.5: 6, 75.0: 4, 87.5: 2})
Combined Emotions
[45.375, 0.3354480935730935]
Arousal 64.625
Counter({75.0: 35, 62.5: 28, 50.0: 16, 37.5: 10, 87.5: 9, 100.0: 1, 25.0: 1})
Valence 69.25
Counter({75.0: 37,

Array of average Happy Sad accurecies for user s15
Counter({50.0: 63, 75.0: 19, 25.0: 15, 0.0: 2, 100.0: 1})
Happy Sad
[50.5, 0.4306190476190477]
Array of average All Emotions accurecies for user s15
Counter({37.5: 44, 25.0: 22, 50.0: 21, 12.5: 6, 62.5: 4, 75.0: 2, 0.0: 1})
Combined Emotions
[37.25, 0.31319146825396826]
Arousal 51.125
Counter({50.0: 30, 62.5: 25, 37.5: 14, 75.0: 12, 25.0: 12, 12.5: 4, 87.5: 2, 100.0: 1})
Valence 58.875
Counter({62.5: 35, 50.0: 27, 75.0: 19, 37.5: 7, 87.5: 6, 25.0: 5, 12.5: 1})
Arousal Valence
[51.125, 58.875, 0.446642440892441, 0.5088682428682428]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 39, 60.0: 27, 100.0: 16, 40.0: 15, 20.0: 2, 0.0: 1})
Happy Sad
[69.8, 0.6611111111111113]
Array of average All Emotions accurecies for user s16
Counter({37.5: 29, 25.0: 24, 50.0: 22, 12.5: 13, 62.5: 9, 0.0: 3})
Combined Emotions
[35.125, 0.2819930555555556]
Arousal 47.0
Counter({37.5: 34, 50.0: 30, 62.5: 14, 25.0: 12, 87.5: 5, 75.0: 4, 12.5

Array of average Happy Sad accurecies for user s29
Counter({83.33333333333334: 34, 66.66666666666666: 28, 100.0: 17, 50.0: 14, 33.33333333333333: 6, 16.666666666666664: 1})
Happy Sad
[73.16666666666667, 0.6608322510822511]
Array of average All Emotions accurecies for user s29
Counter({50.0: 25, 62.5: 25, 37.5: 19, 75.0: 19, 87.5: 6, 25.0: 5, 12.5: 1})
Combined Emotions
[56.125, 0.4435381562881563]
Arousal 67.25
Counter({62.5: 32, 75.0: 30, 50.0: 17, 87.5: 13, 37.5: 5, 100.0: 3})
Valence 58.625
Counter({62.5: 36, 50.0: 24, 75.0: 16, 37.5: 14, 87.5: 7, 25.0: 3})
Arousal Valence
[67.25, 58.625, 0.5828699633699634, 0.5033118548118548]
s30
Array of average Happy Sad accurecies for user s30
Counter({71.42857142857143: 34, 57.14285714285714: 26, 85.71428571428571: 22, 42.857142857142854: 13, 100.0: 3, 28.57142857142857: 2})
Happy Sad
[67.14285714285714, 0.5659502719502719]
Array of average All Emotions accurecies for user s30
Counter({62.5: 40, 50.0: 26, 37.5: 12, 75.0: 10, 25.0: 9, 87.5: 2, 